In [2]:
#auto reload modules
%load_ext autoreload
%autoreload 2

from datetime import datetime
import os
import random
import json
from src.llm.PredictionUtils import init_model_and_tokenizer,llm_gen
from src.prop.utils import gen_reason,gen_masked_prediction_problem_prompts,parse_prediction_with_check

pid = os.getpid()
seed = int(pid)+int(datetime.now().timestamp())
print("seed: ", seed)
random.seed(seed)



out_dir = "data"
os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
seed:  1727714276


In [3]:
model_id="meta-llama/Meta-Llama-3.1-70B-Instruct"
model,tokenizer=init_model_and_tokenizer(model_id)

INFO 09-21 16:55:33 config.py:904] Defaulting to use mp for distributed inference
INFO 09-21 16:55:33 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llam

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


INFO 09-21 16:55:53 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=809061) INFO 09-21 16:55:53 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=809057) INFO 09-21 16:55:54 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=809059) INFO 09-21 16:55:54 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=809058) INFO 09-21 16:55:54 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=809056) INFO 09-21 16:55:55 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=809060) INFO 09-21 16:55:56 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=809055) INFO 09-21 16:55:56 model_runner.py:1008] Loading model weights took 16.4605 GB
INFO 09-21 16:55:59 distributed_gpu_executor.py:57] # GPU blocks: 82648, # CPU blocks: 6553
(VllmWorkerProcess pid=809057) INFO 09-21 16:56:02 

In [6]:
from datasets import load_dataset

ds=load_dataset("kanhatakeyama/material-properties",split="train")
ds=ds.shuffle(seed=1)
len(ds)

7241

In [7]:
train_ds=ds.select(range(7000))
test_ds=ds.select(range(7000,7200))
train_ds[0]

{'CompName': 'unknown',
 'SMILES': 'CC1=CC=C(C=C1)O',
 'Property': 'Vapor pressure',
 'Value': 0.11,
 'unit': '[kPa]',
 'Source': 'Wikipedia/Wikidata'}

In [79]:
import copy
import json

batch_size=100

target_ds=train_ds

def record_to_list_text(record):
    record=copy.deepcopy(record)
    record.pop("Source")
    text=json.dumps(record)
    return text


style_list=[
    "textbook-style",
    "conversation-style",
    "classroom-style",
    "Q&A-style",
    "interview-style",
    "journal-style",
]
prompt_template="""Prepare {} text according to the following data.
- Never include any other number which is not given from the data.
- Generate only the text, not e.g., system outputs.
- Add some discussion or explanation if necessary.
"""

def gen_prompt(tokenizer,target_ds,style_list):
    style=random.choice(style_list)
    q=prompt_template.format(style)

    for i in range(random.randint(1,4)):
        record=target_ds[random.randint(0,len(target_ds)-1)]
        q+=record_to_list_text(record)+"\n"
    
    q=q.strip()
    chat = [
        {"role": "user", "content": q},
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False,)
    assist_prefix = "assistant\n\n"
    prompt += assist_prefix
    return prompt



for i in range(1000):
    prompt_list=[(gen_prompt(tokenizer,target_ds,style_list)) for i in range(batch_size)]
    predicted_text_list=llm_gen(model,prompt_list)

    for i in range(len(predicted_text_list)):
        with open(f"data/{current_time_no_symbols}_llm_gen.jsonl","a", encoding='utf-8') as f:
            f.write(json.dumps({"text":predicted_text_list[i]},ensure_ascii=False)+"\n")


Processed prompts: 100%|██████████| 3/3 [00:09<00:00,  3.23s/it, est. speed input: 82.41 toks/s, output: 102.94 toks/s]
